In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
import re

#got started with code from medium ariticle: https://link.medium.com/siL0vVKhp7

#Get Lis of S&P
companies = pd.read_csv('https://datahub.io/core/s-and-p-500-companies/r/constituents.csv')
Symbol = companies.Symbol

def getfinancialreportingdf(ticker, timeframe = 'A'):


    
    # try:
    if timeframe == 'Q':
        urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/income/quarter'
        urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet/quarter'
        tfIndex = ['MRQ-4', 'MRQ-3', 'MRQ-2', 'MRQ-1', 'MRQ']
    else:
        urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
        urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
        tfIndex = range(date.today().year-5,date.today().year)
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml") #read in
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml") #read in


    # Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    stdebtlist = []
    interestexpenselist = []
    ebitdalist= []
    cashlist = []
    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

    

    # Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'ST Debt & Current Portion LT Debt' in title.text:
            stdebtlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

        if 'Cash' in title.text:
            cashlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
            
    eps = getelementinlist(epslist,0)
    epsgrowth = getelementinlist(epslist,1)
    netincome = getelementinlist(netincomelist,0)
    shareholderequity = getelementinlist(equitylist,0)
    roa = getelementinlist(equitylist,1)
    cash = getelementinlist(cashlist,0)
    
    
    longtermdebt = getelementinlist(longtermdebtlist,0)
    stdebt = getelementinlist(stdebtlist,0)
    interestexpense =  getelementinlist(interestexpenselist,0)
    ebitda = getelementinlist(ebitdalist,0)
    # Don't forget to add in roe, interest coverage ratio

    ## Make it into Dataframes

    df= pd.DataFrame({'eps': eps,'epsgrowth': epsgrowth,'netincome': netincome,'shareholderequity': shareholderequity,'roa': 
                  roa,'longtermdebt': longtermdebt,'interestexpense': interestexpense,'ebitda': ebitda, 'ST Debt': stdebt,
                     'Cash': cash},
                      index=tfIndex)
    return df

def getelementinlist(list,element):
    try:
        return list[element]
    except:
        return 0





<font color=red> **Note: each loop takes approximately 10 to 15 minutes to complete. The code will print out what stock you're on in the output i.e. '1 out of 505'. You can just let run in the background. </font>

In [25]:
# run quarterly financials

SPListQ = [] 
counter = 0

for ticker in Symbol:
    counter = counter + 1

    try:
        df = getfinancialreportingdf(ticker, timeframe = 'Q')
        df['Ticker'] = ticker
        SPListQ.append(df)
        print(str(counter) + ' of ' + str(len(Symbol)))
    except:
        print('Error Downloading: ' + ticker)
        





1 of 505
2 of 505
3 of 505
4 of 505
5 of 505


In [28]:
# run annual financials

SPList = [] 
counter = 0

for ticker in Symbol:
    counter = counter + 1

    try:
        df = getfinancialreportingdf(ticker)
        df['Ticker'] = ticker
        SPList.append(df)
        print(str(counter) + ' of ' + str(len(Symbol)))
    except:
        print('Error Downloading: ' + ticker)
        

1 of 505
2 of 505
3 of 505
4 of 505
5 of 505


In [24]:
StockDF.to_csv('Stock_Scrape.csv')
StockDFQ.to_csv('Stock_ScrapeQ.csv')